# Prediction with Sub-Catchment Areas For Each Site

## Imports

In [1]:
import pandas as pd
import prediction_of_H_indicator_with_subCatchmentData as prediction
import ipywidgets as widgets
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

## Variables

In [2]:
RESULT_FOLDER = "data"

## Import Data and Clean them

In [3]:
input_data = prediction.import_input_data_clean()
input_data

,Site,SubCatch,Slope,Elevation,LC,SAR,Area,CV,HV,HError,Rate
0,1,1,1.203898,8.936689,319.676414,1.000254,1648125,167,211,0.000000,1.0
1,1,1,1.203898,8.936689,319.676414,1.000254,1648125,167,211,NaN,2.0
2,1,1,1.203898,8.936689,319.676414,1.000254,1648125,167,211,NaN,7.0
3,1,1,1.203898,8.936689,319.676414,1.000254,1648125,167,211,NaN,15.0
4,1,1,1.203898,8.936689,319.676414,1.000254,1648125,167,211,0.035656,21.0
...,...,...,...,...,...,...,...,...,...,...,...
6770,40,15,8.875044,224.490265,933.385135,1.006683,4387500,0,12,NaN,NaN
6771,40,16,6.585227,178.075058,700.135624,1.003876,2930625,0,20,NaN,NaN
6772,40,17,5.182087,173.802765,685.592278,1.002990,3661875,0,34,NaN,NaN
6773,40,18,4.617903,196.079285,597.577939,1.002033,5023125,0,68,NaN,NaN


In [4]:
input_data_cleaned = prediction.clean_data(input_data)
input_data_cleaned

,Site,SubCatch,Slope,Elevation,LC,SAR,Area,CV,HV,HError,Rate
0,1,1,1.203898,8.936689,319.676414,1.000254,1648125,167,211,0.000000,1.0
4,1,1,1.203898,8.936689,319.676414,1.000254,1648125,167,211,0.035656,21.0
8,1,1,1.203898,8.936689,319.676414,1.000254,1648125,167,211,0.083538,60.0
9,1,1,1.203898,8.936689,319.676414,1.000254,1648125,167,211,0.093477,75.0
11,1,1,1.203898,8.936689,319.676414,1.000254,1648125,167,211,0.116415,100.0
...,...,...,...,...,...,...,...,...,...,...,...
6603,39,4,8.758309,162.546722,690.708947,1.006050,2925000,0,27,0.000000,1.0
6633,39,5,7.371158,150.872406,470.062314,1.004910,8943750,0,127,0.000000,1.0
6664,39,7,4.720726,195.869247,672.920859,1.001445,1299375,0,5,0.000000,1.0
6694,39,8,6.162337,121.253716,589.868107,1.003319,9225000,0,61,0.000000,1.0


## Choosing the new site to make predictions for

In [5]:
# CAUTION : if running from jupyterlba => jupyter labextension install @jupyter-widgets/jupyterlab-manager
## IF this cell is not running => SKIP THIS CELL and GO TO NEXT CELL :D


from IPython.display import display, Javascript

drop = widgets.Dropdown(
    options=[('Breville-Sur-Mer', 1), ('Agon-Coutainville', 2), ('Saint-Germain-Sur-Ay', 3), ('Jullouville', 4), ('Etréham', 5), ('Genêts', 6), ('Saint-Pair-sur-Mer', 7), ('Arromanches-les-Bains', 8), ('Port-en-Bessin', 9), ('Courtils', 10), ('Lessay', 11), ('Doville', 12), ('Graye-sur-Mer', 13), ('Saint-Potan', 14), ('Saint-Vaast-la-Hougue', 15), ('La_Pernelle', 16), ('Lestre', 17), ('Banville', 18), ('Isigny-sur-Mer', 19), ('Saint-Malo', 20), ('Pierreville', 21), ('La_Feuillie', 22), ('Tourlaville', 23), ('Octeville', 24), ('Saint-Briac-sur-Mer', 25), ('Granville', 26), ('Blainville-sur-Mer', 27), ('Hauteville-sur-Mer', 28), ('Sainte-Marie-du-Mont', 29), ('Vierville-sur-Mer', 30), ('Cherrueix', 31), ('Chef-du-Pont', 32), ('Saint-Lô', 33), ('Couvains', 34), ('Rocheville', 35), ('Lison', 36), ('Marigny', 37), ('Carville', 38), ('Percy', 39), ('Saint-Armand', 40)],
    value=7,
    description='Site:',
)
button = widgets.Button(description="Valid")
output = widgets.Output()

display(drop, button, output)

def on_button_clicked(b):
    global site_number
    site_number = drop.value

button.on_click(on_button_clicked)

Dropdown(description='Site:', index=6, options=(('Breville-Sur-Mer', 1), ('Agon-Coutainville', 2), ('Saint-Ger…

Button(description='Valid', style=ButtonStyle())

Output()

In [7]:
# Manually change the site to predict the H values for ;p
site_number = 5

## Prepare datasets for training the prediction model

In [5]:
features, variable_to_predict = prediction.split_dataset_into_features_and_variable_to_predict(input_data_cleaned)
#print(features, variable_to_predict)

In [6]:
def get_train_and_test_data_for_a_subcatch(X, y, test_site, test_subcatch):
    y_test = y.loc[(y.Site == test_site) & (y.SubCatch == test_subcatch)]
        ## We do not want to take the site number into account for the prediction
    del y_test["Site"]
    del y_test["SubCatch"]

        # Removing the data for the site we want to predict  =>>> /!\ DONE : remove only the test subcatch of the test site !
    #print(y[y.Site == test_site].index)
    #print(y.loc[(y.Site == test_site) & (y.SubCatch == test_subcatch)].index)
    y_train = y.drop(y.loc[(y.Site == test_site)].index)
        ## We do not want to take the site number into account for the prediction
    del y_train["Site"]
    del y_train["SubCatch"]

        # Splitting the x (features) into training and testing data
    X_test = X.loc[(X.Site == test_site) & (X.SubCatch == test_subcatch)]
        ## We do not want to take the site number into account for the prediction
    del X_test["Site"]
    del X_test["SubCatch"]

        # Removing the data for the site we want to predict
    X_train = X.drop(X.loc[(X.Site == test_site)].index)
        ## We do not want to take the site number into account for the prediction
    del X_train["Site"]
    del X_train["SubCatch"]

    
    return X_train, X_test, y_train, y_test

In [10]:
subcatch_number = 4
features_train, features_test, variable_train, variable_test = get_train_and_test_data_for_a_subcatch(features, variable_to_predict, site_number, subcatch_number)
print(features_test, variable_test)

       Slope  Elevation          LC      SAR     Area  CV  HV    Rate
629  4.36158  43.982922  817.560286  1.00163  4471875  35  54     1.0
630  4.36158  43.982922  817.560286  1.00163  4471875  35  54     2.0
631  4.36158  43.982922  817.560286  1.00163  4471875  35  54     7.0
632  4.36158  43.982922  817.560286  1.00163  4471875  35  54    15.0
633  4.36158  43.982922  817.560286  1.00163  4471875  35  54    21.0
634  4.36158  43.982922  817.560286  1.00163  4471875  35  54    30.0
635  4.36158  43.982922  817.560286  1.00163  4471875  35  54    45.0
636  4.36158  43.982922  817.560286  1.00163  4471875  35  54    50.0
637  4.36158  43.982922  817.560286  1.00163  4471875  35  54    60.0
638  4.36158  43.982922  817.560286  1.00163  4471875  35  54    75.0
639  4.36158  43.982922  817.560286  1.00163  4471875  35  54    90.0
640  4.36158  43.982922  817.560286  1.00163  4471875  35  54   100.0
641  4.36158  43.982922  817.560286  1.00163  4471875  35  54   125.0
642  4.36158  43.982

## Training prediction model

In [11]:
def train_random_forest_model(X_train, y_train):
    forest = RandomForestRegressor(
        n_estimators=1000, criterion="mse", random_state=1, n_jobs=-1, oob_score = True, bootstrap = True
    )
    forest.fit(X_train, y_train.values.ravel())
    return forest

prediction_model = train_random_forest_model(features_train, variable_train)

In [12]:
features_train

,Slope,Elevation,LC,SAR,Area,CV,HV,Rate
0,1.203898,8.936689,319.676414,1.000254,1648125,167,211,1.0
4,1.203898,8.936689,319.676414,1.000254,1648125,167,211,21.0
8,1.203898,8.936689,319.676414,1.000254,1648125,167,211,60.0
9,1.203898,8.936689,319.676414,1.000254,1648125,167,211,75.0
11,1.203898,8.936689,319.676414,1.000254,1648125,167,211,100.0
...,...,...,...,...,...,...,...,...
6603,8.758309,162.546722,690.708947,1.006050,2925000,0,27,1.0
6633,7.371158,150.872406,470.062314,1.004910,8943750,0,127,1.0
6664,4.720726,195.869247,672.920859,1.001445,1299375,0,5,1.0
6694,6.162337,121.253716,589.868107,1.003319,9225000,0,61,1.0


In [12]:
#prediction_model.feature_importances_    /!\ USE permutation importance!

## Prediction

In [13]:
def predict_with_trained_model(forest, X_train, X_test):
    # Predicting results
    y_train_pred = forest.predict(X_train)
    y_test_pred = forest.predict(X_test)
    return y_train_pred, y_test_pred

variable_train_pred, variable_test_pred = predict_with_trained_model(prediction_model, features_train, features_test)

## Quality metrics for H values (MSE & NSE/R² scores)

In [14]:
subCatchment_numbers = prediction.get_subcatchment_numbers_for_a_subcatch(site_number, variable_to_predict, subcatch_number)
liste_variable_test_HError = prediction.get_list_variable_test_Hind_Values(variable_test)
liste_variable_test_pred_HError = variable_test_pred
#print(subCatchment_numbers, liste_variable_test_HError, liste_variable_test_pred_HError)
mse_test, r2_test = prediction.get_standard_quality_metrics(subCatchment_numbers, liste_variable_test_HError, liste_variable_test_pred_HError)
print(mse_test, r2_test)

{4: 0.014288007981789507} {4: -5.363992450388677}


## Get Pmax real and Pmax predicted

In [15]:
features

,Site,SubCatch,Slope,Elevation,LC,SAR,Area,CV,HV,Rate
0,1,1,1.203898,8.936689,319.676414,1.000254,1648125,167,211,1.0
4,1,1,1.203898,8.936689,319.676414,1.000254,1648125,167,211,21.0
8,1,1,1.203898,8.936689,319.676414,1.000254,1648125,167,211,60.0
9,1,1,1.203898,8.936689,319.676414,1.000254,1648125,167,211,75.0
11,1,1,1.203898,8.936689,319.676414,1.000254,1648125,167,211,100.0
...,...,...,...,...,...,...,...,...,...,...
6603,39,4,8.758309,162.546722,690.708947,1.006050,2925000,0,27,1.0
6633,39,5,7.371158,150.872406,470.062314,1.004910,8943750,0,127,1.0
6664,39,7,4.720726,195.869247,672.920859,1.001445,1299375,0,5,1.0
6694,39,8,6.162337,121.253716,589.868107,1.003319,9225000,0,61,1.0


In [21]:
rates = prediction.get_rates_for_a_subcatch(site_number, features, subcatch_number)
liste_variable_test_HError = prediction.get_list_variable_test_Hind_Values(variable_test)
liste_variable_test_pred_HError = variable_test_pred
subcatch = 4
p_test = 0
p_pred = 0
H_limit = 0.1
pmaxTest_found = False
pmaxPred_found = False

print(rates, len(rates))
print(liste_variable_test_HError, len(liste_variable_test_HError))
print(liste_variable_test_pred_HError, len(liste_variable_test_pred_HError))

for index_sub in range(len(liste_variable_test_HError)):

    if pmaxTest_found is False and liste_variable_test_HError[index_sub] > H_limit:
        p_test = rates[index_sub -1]
        pmaxTest_found = True
    elif pmaxTest_found is False and index_sub == len(liste_variable_test_HError)-1:
        if rates[-1] == float(3652):
            p_test = 3652
    elif pmaxTest_found:
        print(index_sub)
        break
            
for index_sub in range(len(liste_variable_test_pred_HError)):    
    if pmaxPred_found is False and liste_variable_test_pred_HError[index_sub] > H_limit:
        p_pred = rates[index_sub -1]
        pmaxPred_found = True
    elif pmaxPred_found is False and index_sub == len(liste_variable_test_pred_HError)-1:
        if rates[-1] == float(3652):
            p_pred = rates[-1]
    elif pmaxPred_found:
        print(index_sub)
        break

print("Real value of p: ", p_test)
print("Predicted value of p: ", p_pred)

[1.0, 2.0, 7.0, 15.0, 21.0, 30.0, 45.0, 50.0, 60.0, 75.0, 90.0, 100.0, 125.0, 150.0, 182.0, 200.0, 250.0, 300.0, 330.0, 365.0, 550.0, 640.0, 730.0, 1000.0, 1500.0, 2000.0, 2250.0, 3000.0, 3182.0, 3652.0] 30
[0.0, 0.002526966035067, 0.009512015447923, 0.016052916554261, 0.018748325858716, 0.023852255699491, 0.031747779827179, 0.032877687915139, 0.038381191284593, 0.043166721775508, 0.053638078472765, 0.056545617622844, 0.069232078569536, 0.078635560739383, 0.103604689122397, 0.094220461256899, 0.110428507953097, 0.116813838960434, 0.116448998394691, 0.116676267527529, 0.123356306885072, 0.122262117667871, 0.127822366652954, 0.126051820731634, 0.129826277701743, 0.132835415015362, 0.130450582704871, 0.135461041100975, 0.132959066467299, 0.130211005584775] 30
[0.         0.00344569 0.01582923 0.02294714 0.02540554 0.03222838
 0.0514403  0.054902   0.06078319 0.07061313 0.09426985 0.09591724
 0.12172878 0.15472735 0.19328758 0.18871854 0.20724732 0.21559709
 0.21340844 0.21690003 0.2714715

In [16]:
rates = prediction.get_rates_for_a_subcatch(site_number, features, subcatch_number)
pmax_test, pmax_pred = prediction.get_real_and_pred_pmax(subCatchment_numbers, rates, liste_variable_test_HError, variable_test_pred)
print(pmax_test, pmax_pred)

{4: 150.0} {4: 100.0}


## Save results into files

In [17]:
prediction.save_Hind_results_into_file_for_a_subcatch(site_number, subcatch_number, 1, subCatchment_numbers, rates, liste_variable_test_HError, variable_test_pred, approx=0, chronicle=0, permeability=27.32)
prediction.save_Pmax_results_into_file_for_a_subcatch(site_number, subcatch_number, 1, pmax_test, pmax_pred, mse_test, r2_test, approx=0, chronicle=0, permeability=27.32)

File created here:  output/Approx0/Chronicle0/SiteTest5/Prediction_HErrorValues_SubCatch4_Chronicle0_Approx0_K27.32_Slope_Elevation_LC_SAR_Area_CV_HV_clean.csv
File created here: output/Approx0/Chronicle0/SiteTest5/Prediction_PMax_SubCatch4_Chronicle0_Approx0_K27.32_Slope_Elevation_LC_SAR_Area_CV_HV_clean.csv


# Prediction Workflow

In [1]:
import pandas as pd
import prediction_of_H_indicator_with_subCatchmentData as prediction
import ipywidgets as widgets
import numpy as np
import csv
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.inspection import permutation_importance

def get_feature_importance_for_a_site_and_a_subcatch_no_CV(prediction_model, features_test, site_number, subcatch_number, nb_clusters=1, approx=0, chronicle=0, permeability=27.32):
    sort_indexes_imp = np.argsort(prediction_model.feature_importances_)
    #print(sort_indexes_imp[::-1])
    
    MYDIR = (
        "output"
        + "/"
        + "Approx"
        + str(approx)
        + "/Chronicle"
        + str(chronicle)
        + "/SiteTest"
        + str(site_number)
    )
    with open(
        MYDIR
        + "/"
        + "Feature_Importance"
        + "_Site"
        + str(site_number)
        + "_SubCatch"
        + str(subcatch_number)
        +"_Chronicle"
        + str(chronicle)
        + "_Approx"
        + str(approx)
        + "_K"
        + str(permeability)
        + "_Slope_Elevation_LC_SAR_Area_HV" #_CV_HV
        + "Clusters" + str(nb_clusters) + "_clean.csv",
        "w",
    ) as f:
        writer = csv.writer(f, delimiter=";")
        writer.writerow(
            [
                "Approx",
                "Chronicle",
                "Test Site",
                "SubCatchment",
                "Feature",
                "ImportanceValue",
                "ImportanceRank",
            ]
        )
        rank=0
        for i in sort_indexes_imp[::-1]:
            rank+=1
            writer.writerow(
                [
                    approx,
                    chronicle,
                    site_number,
                    subcatch_number,
                    features_test.columns[i],
                    prediction_model.feature_importances_[i],
                    rank,
                ]
            )



def prediction_worflow_for_a_site_and_subcatch(site_number, subcatch_number):
    print("----Site", site_number, "Sub Catch", subcatch_number, "----")
    # Import input dataset & clean it
    input_data = prediction.import_input_data_clean()
    input_data_cleaned = prediction.clean_data(input_data)
    # Split dataset
    features, variable_to_predict = prediction.split_dataset_into_features_and_variable_to_predict(input_data_cleaned)
    features_train, features_test, variable_train, variable_test = prediction.get_train_and_test_data_for_a_subcatch_no_CV(features, variable_to_predict, site_number, subcatch_number)
    # Prediction
    prediction_model = prediction.train_random_forest_model(features_train, variable_train)
    variable_train_pred, variable_test_pred = prediction.predict_with_trained_model(prediction_model, features_train, features_test)
    # Permutation Importance
    #print("feature importance", prediction_model.feature_importances_)
    get_feature_importance_for_a_site_and_a_subcatch_no_CV(prediction_model, features_test, site_number, subcatch_number)

    
    #get_permut_importance(prediction_model, features_test, variable_test, site_number, subcatch_number)
    
    subCatchment_numbers = prediction.get_subcatchment_numbers_for_a_subcatch(site_number, variable_to_predict, subcatch_number)
    liste_variable_test_HError = prediction.get_list_variable_test_Hind_Values(variable_test)
    liste_variable_test_pred_HError = variable_test_pred
    mse_test, r2_test = prediction.get_standard_quality_metrics(subCatchment_numbers, liste_variable_test_HError, liste_variable_test_pred_HError)
    print(mse_test, r2_test)
    rates = prediction.get_rates_for_a_subcatch(site_number, features, subcatch_number)
    pmax_test, pmax_pred = prediction.get_real_and_pred_pmax(subCatchment_numbers, rates, liste_variable_test_HError, variable_test_pred)
    print(pmax_test, pmax_pred)
    prediction.save_Hind_results_into_file_for_a_subcatch_no_CV(site_number, subcatch_number, 1, subCatchment_numbers, rates, liste_variable_test_HError, variable_test_pred, approx=0, chronicle=0, permeability=27.32)
    prediction.save_Pmax_results_into_file_for_a_subcatch_no_CV(site_number, subcatch_number, 1, pmax_test, pmax_pred, mse_test, r2_test, approx=0, chronicle=0, permeability=27.32)

In [ ]:
for site_number in range(1,41):
    for sub in range(1, 40):
        try:
            prediction_worflow_for_a_site_and_subcatch(site_number, sub)
        except:
            continue

In [56]:
def get_features_imp_for_a_site_and_all_subcatchs(site_number, nb_clusters=1, approx=0, chronicle=0, permeability=27.32):

    imp_features_df_all = pd.DataFrame(columns=["Approx",
                "Chronicle",
                "Test Site",
                "SubCatchment",
                "Feature",
                "ImportanceValue",
                "ImportanceRank"])
    for subcatch_number in range(1, 40):
        MYDIR = (
            "output"
            + "/"
            + "Approx"
            + str(approx)
            + "/Chronicle"
            + str(chronicle)
            + "/SiteTest"
            + str(site_number)
        )
        file = (MYDIR
        + "/"
        + "Feature_Importance"
        + "_Site"
        + str(site_number)
        + "_SubCatch"
        + str(subcatch_number)
        +"_Chronicle"
        + str(chronicle)
        + "_Approx"
        + str(approx)
        + "_K"
        + str(permeability)
        + "_Slope_Elevation_LC_SAR_Area_HV" #_CV_HV
        + "Clusters" + str(nb_clusters) + "_clean.csv")
        try:
            imp_sub = pd.read_csv(file, sep=";")
            imp_features_df_all = pd.concat([imp_features_df_all, imp_sub])
        except:
            continue
    #print(imp_features_df_all)
    
    out_file =("Feature_Importance" + "_Site"
        + str(site_number)
        + "_All_SubCatchs"
        +"_Chronicle"
        + str(chronicle)
        + "_Approx"
        + str(approx)
        + "_K"
        + str(permeability)
        + "_Slope_Elevation_LC_SAR_Area_HV" #_CV_HV
        + "Clusters" + str(nb_clusters) + "_clean.csv")
    imp_features_df_all.to_csv(MYDIR + "/" + out_file, sep=";", index=False)
    return imp_features_df_all

#get_features_imp_for_a_site_and_all_subcatchs(5)

In [55]:
imp_features_df_all = get_features_imp_for_a_site_and_all_subcatchs(3)
imp_features_df_all.to_csv("output/" + "test.csv", sep=";", index=False)

,Approx,Chronicle,Test Site,SubCatchment,Feature,ImportanceValue,ImportanceRank
0,0,0,5,1,Elevation,0.301368,1
1,0,0,5,1,Rate,0.257340,2
2,0,0,5,1,LC,0.149976,3
3,0,0,5,1,SAR,0.089709,4
4,0,0,5,1,Area,0.081249,5
...,...,...,...,...,...,...,...
3,0,0,5,17,SAR,0.083881,4
4,0,0,5,17,Area,0.070660,5
5,0,0,5,17,Slope,0.066038,6
6,0,0,5,17,HV,0.048680,7


In [57]:
for site in range(1, 41):
    try:
        get_features_imp_for_a_site_and_all_subcatchs(site)
    except:
        continue
print("Finished.")

Finished.


In [25]:
def get_permut_importance(prediction_model, features_test, variable_test, site_number, subcatch_number):
    permut = permutation_importance(prediction_model, features_test, variable_test, n_repeats=10, random_state=42)
    print("permut", permut.importances_mean)
    sorted_idx = permut.importances_mean.argsort()
    fig, ax = plt.subplots()
    ax.boxplot(permut.importances[sorted_idx].T,
           vert=False, labels=features_test.columns[sorted_idx])
    ax.set_title("Permutation Importances (test set)")
    fig.tight_layout()
    #plt.show()
    plt.savefig("output/PermutationImportance/" + "permut_imp_Site" + str(site_number) + "_SubCatch" + str(subcatch_number)  +".jpg")
    

In [62]:
def predition_workflow_for_all_sites_and_all_their_subcatchs():
    for site_number in range(1,41):
        prediction_workflow_for_a_site_for_all_its_subcatchs(site_number)    
    print("Predictions are finished.")

In [ ]:
predition_workflow_for_all_sites_and_all_their_subcatchs()

In [ ]:
for site_nb in [16, 19, 20, 21, 22,25,27,28,29,33,34,35,37,38,39]:
    prediction_workflow_for_a_site_for_all_its_subcatchs(site_nb)

# Concatenate the prediction results for all the sites and all the subcatchs

In [15]:
import pandas as pd

def concat_prediction_pmax_results_with_subcatchment_for_a_site_and_all_its_subcatchs(site_number, nb_clusters=1, approx=0, chronicle=0, permeability=27.32):
    print("---- Site", site_number, "----")
    frames = []
    path = ("output"
            + "/"
            + "Approx"
            + str(approx)
            + "/Chronicle"
            + str(chronicle)
            + "/SiteTest"
            + str(site_number)
            + "/"
    )
    for subcatch_number in range(1,31):
        file = "Prediction_PMax_SubCatch" + str(subcatch_number) + "_Chronicle" + str(chronicle) + "_Approx" + str(approx) + "_K" + str(permeability) + "_Slope_Elevation_LC_SAR_Area_CV_HV" + "Clusters" + str(nb_clusters) + "_clean.csv"
        try:
            dfp = pd.read_csv(path + file, sep=";")
            print(dfp)
            frames.append(dfp)
        except:
            continue
        
    #print(frames)
    df = pd.concat(frames)
    df.to_csv(path + "Prediction_PMax_All_SubCatchs_Chronicle" + str(chronicle) + "_Approx"
                    + str(approx)
                    + "_K"
                    + str(permeability) + "_Slope_Elevation_LC_SAR_Area_HV" + "_Clusters" + str(nb_clusters) + "_clean.csv", index=False, sep=";")
    print(
        "File '"
        + "Prediction_PMax_All_SubCatchs_Chronicle" + str(chronicle) + "_Approx"
        + str(approx)
        + "_K"
        + str(permeability) + "_Slope_Elevation_LC_SAR_Area_HV_clean.csv"
        + "' has been created.")
    

def concat_prediction_h_results_with_subcatchment_for_a_site_and_all_its_subcatchs(site_number, nb_clusters=1, approx=0, chronicle=0, permeability=27.32):
    print("---- Site", site_number, "----")
    frames = []
    path = ("output"
            + "/"
            + "Approx"
            + str(approx)
            + "/Chronicle"
            + str(chronicle)
            + "/SiteTest"
            + str(site_number)
            + "/"
    )
    for subcatch_number in range(1,31):
        file = "Prediction_HErrorValues_SubCatch" + str(subcatch_number) + "_Chronicle" + str(chronicle) + "_Approx" + str(approx) + "_K" + str(permeability) + "_Slope_Elevation_LC_SAR_Area_CV_HV" + "Clusters" + str(nb_clusters) + "_clean.csv"
        try:
            dfp = pd.read_csv(path + file, sep=";")
            #print(dfp)
            frames.append(dfp)
        except:
            continue
        
    #print(frames)
    df = pd.concat(frames)
    df.to_csv(path + "Prediction_HErrorValues_All_SubCatchs_Chronicle" + str(chronicle) + "_Approx"
                    + str(approx)
                    + "_K"
                    + str(permeability) + "_Slope_Elevation_LC_SAR_Area_CV_HV" + "_Clusters" + str(nb_clusters) + "_clean.csv", index=False, sep=";")
    print(
        "File '"
        + "Prediction_HErrorValues_All_SubCatchs_Chronicle" + str(chronicle) + "_Approx"
        + str(approx)
        + "_K"
        + str(permeability) + "_Slope_Elevation_LC_SAR_Area_HV_clean.csv"
        + "' has been created."
    )

In [16]:
def concat_prediction_pmax_results_with_subcatchment_for_all_sites_by_site_and_all_its_subcatchs(nb_clusters=1, approx=0, chronicle=0, permeability=27.32):
    for site_number in range(1,41):
        try:
            concat_prediction_pmax_results_with_subcatchment_for_a_site_and_all_its_subcatchs(site_number, nb_clusters, approx, chronicle, permeability)
        except:
            continue
    print("Concatenations finished for all sites.")

def concat_prediction_h_results_with_subcatchment_for_all_sites_by_site_and_all_its_subcatchs(nb_clusters=1, approx=0, chronicle=0, permeability=27.32):
    for site_number in range(1,41):
        try:
            concat_prediction_h_results_with_subcatchment_for_a_site_and_all_its_subcatchs(site_number, nb_clusters, approx, chronicle, permeability)
        except:
            continue
    print("Concatenations finished for all sites.")

In [17]:
concat_prediction_h_results_with_subcatchment_for_all_sites_by_site_and_all_its_subcatchs()

---- Site 1 ----
File 'Prediction_HErrorValues_All_SubCatchs_Chronicle0_Approx0_K27.32_Slope_Elevation_LC_SAR_Area_HV_clean.csv' has been created.
---- Site 2 ----
File 'Prediction_HErrorValues_All_SubCatchs_Chronicle0_Approx0_K27.32_Slope_Elevation_LC_SAR_Area_HV_clean.csv' has been created.
---- Site 3 ----
File 'Prediction_HErrorValues_All_SubCatchs_Chronicle0_Approx0_K27.32_Slope_Elevation_LC_SAR_Area_HV_clean.csv' has been created.
---- Site 4 ----
---- Site 5 ----
File 'Prediction_HErrorValues_All_SubCatchs_Chronicle0_Approx0_K27.32_Slope_Elevation_LC_SAR_Area_HV_clean.csv' has been created.
---- Site 6 ----
File 'Prediction_HErrorValues_All_SubCatchs_Chronicle0_Approx0_K27.32_Slope_Elevation_LC_SAR_Area_HV_clean.csv' has been created.
---- Site 7 ----
File 'Prediction_HErrorValues_All_SubCatchs_Chronicle0_Approx0_K27.32_Slope_Elevation_LC_SAR_Area_HV_clean.csv' has been created.
---- Site 8 ----
File 'Prediction_HErrorValues_All_SubCatchs_Chronicle0_Approx0_K27.32_Slope_Elevatio

In [18]:
def concat_prediction_pmax_results_with_subcatchment_for_all_sites_and_all_subcatchs_into_one_file(nb_clusters=1, approx=0, chronicle=0, permeability=27.32):
        
    frames = []
   
    for site_number in range(1,41):
        path = ("output"
            + "/"
            + "Approx"
            + str(approx)
            + "/Chronicle"
            + str(chronicle)
            + "/SiteTest"
            + str(site_number)
            + "/"
            )
        file = "Prediction_PMax_All_SubCatchs_Chronicle" + str(chronicle) + "_Approx" + str(approx) + "_K" + str(permeability) + "_Slope_Elevation_LC_SAR_Area_CV_HV" + "_Clusters" + str(nb_clusters) + "_clean.csv"
        try:
            dfp = pd.read_csv(path + file, sep=";")
            frames.append(dfp)
        except:
            continue
        
    df = pd.concat(frames)
    df.to_csv("output/" + "Prediction_PMax_All_Sites_All_SubCatchs_Chronicle" + str(chronicle) + "_Approx"
                    + str(approx)
                    + "_K"
                    + str(permeability) + "_Slope_Elevation_LC_SAR_Area_CV_HV" + "_Clusters" + str(nb_clusters) + "_clean.csv", index=False, sep=";")
    print(
        "File '"
        + "Prediction_PMax_All_Sites_All_SubCatchs_Chronicle" + str(chronicle) + "_Approx"
        + str(approx)
        + "_K"
        + str(permeability) + "_Slope_Elevation_LC_SAR_Area_CV_HV_clean.csv"
        + "' has been created."
    )

def concat_prediction_h_results_with_subcatchment_for_all_sites_and_all_subcatchs_into_one_file(nb_clusters=1, approx=0, chronicle=0, permeability=27.32):
        
    frames = []
   
    for site_number in range(1,41):
        path = ("output"
            + "/"
            + "Approx"
            + str(approx)
            + "/Chronicle"
            + str(chronicle)
            + "/SiteTest"
            + str(site_number)
            + "/"
            )
        file = "Prediction_HErrorValues_All_SubCatchs_Chronicle" + str(chronicle) + "_Approx" + str(approx) + "_K" + str(permeability) + "_Slope_Elevation_LC_SAR_Area_CV_HV" + "_Clusters" + str(nb_clusters) + "_clean.csv"
        try:
            dfp = pd.read_csv(path + file, sep=";")
            frames.append(dfp)
        except:
            continue
        
    df = pd.concat(frames)
    df.to_csv("output/" + "Prediction_HErrorValues_All_Sites_All_SubCatchs_Chronicle" + str(chronicle) + "_Approx"
                    + str(approx)
                    + "_K"
                    + str(permeability) + "_Slope_Elevation_LC_SAR_Area_CV_HV" + "_Clusters" + str(nb_clusters) + "_clean.csv", index=False, sep=";")
    print(
        "File '"
        + "Prediction_HErrorValues_All_Sites_All_SubCatchs_Chronicle" + str(chronicle) + "_Approx"
        + str(approx)
        + "_K"
        + str(permeability) + "_Slope_Elevation_LC_SAR_Area_CV_clean.csv"
        + "' has been created."
    )

In [8]:
concat_prediction_pmax_results_with_subcatchment_for_all_sites_and_all_subcatchs_into_one_file()

File 'Prediction_PMax_All_Sites_All_SubCatchs_Chronicle0_Approx0_K27.32_Slope_Elevation_LC_SAR_Area_CV_clean.csv' has been created.


In [19]:
concat_prediction_h_results_with_subcatchment_for_all_sites_and_all_subcatchs_into_one_file()

File 'Prediction_HErrorValues_All_Sites_All_SubCatchs_Chronicle0_Approx0_K27.32_Slope_Elevation_LC_SAR_Area_CV_clean.csv' has been created.


# Permutation Importance

https://scikit-learn.org/stable/auto_examples/inspection/plot_permutation_importance.html#sphx-glr-auto-examples-inspection-plot-permutation-importance-py

In [ ]:
from sklearn.inspection import permutation_importance
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.ensemble import RandomForestRegressor

def get_fig_perm_import(approx, chronicle, result, test_site, X_test, nse_test):
    sorted_idx = result.importances_mean.argsort()
    fig, ax = plt.subplots()
    ax.boxplot(result.importances[sorted_idx].T,
           vert=False, labels=X_test.columns[sorted_idx])
    ax.set_title("Permutation Importances (test set)")
    fig.tight_layout()
    plt.savefig("output/PermutationImportance/" + "/ZLearning/Approx" + str(approx) + "/Chronicle" + str(chronicle) + "/PermutationImportance_NbFeatures" + str(len(X_test.columns)) + "_TestSite" + str(test_site) + "_NSE" + str(nse_test) + ".jpg")
    
    
    
def permutation_importance_for_a_site(approx, chronicle, test_site, X, y):

    X_train, X_test, y_train, y_test = get_train_and_test_data(X, y, test_site)
    
    # Build a forest and compute the feature importances
    forest = RandomForestRegressor(n_estimators=1000, criterion="mse", random_state=1, n_jobs=-1)
    forest.fit(X_train, y_train.values.ravel())
    
    print("RF train accuracy: %0.3f" % forest.score(X_train, y_train))
    print("RF test accuracy: %0.3f" % forest.score(X_test, y_test))
    nse_test = round(forest.score(X_test, y_test), 3)

    

    result = permutation_importance(forest, X_test, y_test, n_repeats=10,
                                random_state=42, n_jobs=2)
    
    get_fig_perm_import(approx, chronicle, result, test_site, X_test, nse_test)

## Feature Importance : https://towardsdatascience.com/explaining-feature-importance-by-example-of-a-random-forest-d9166011959e

In [15]:
def imp_df(column_names, importances):
    df = pd.DataFrame({'feature': column_names, 'feature_importance': importances}).sort_values('feature_importance', ascending = False).reset_index(drop = True)
    return df

In [16]:
from sklearn.base import clone 

def drop_col_feat_imp(model, X_train, y_train, random_state = 1):
    
    # clone the model to have the exact same specification as the one initially trained
    model_clone = clone(model)
    # set random_state for comparability
    model_clone.random_state = random_state
    # training and scoring the benchmark model
    model_clone.fit(X_train, y_train)
    benchmark_score = model_clone.score(X_train, y_train)
    # list for storing feature importances
    importances = []
    
    # iterating over all columns and storing feature importance (difference between benchmark and new model)
    for col in X_train.columns:
        model_clone = clone(model)
        model_clone.random_state = random_state
        model_clone.fit(X_train.drop(col, axis = 1), y_train)
        drop_col_score = model_clone.score(X_train.drop(col, axis = 1), y_train)
        importances.append(benchmark_score - drop_col_score)
    
    importances_df = imp_df(X_train.columns, importances)
    return importances_df

In [17]:
importances_df = drop_col_feat_imp(prediction_model, features_train, variable_train.values.ravel(), random_state = 1)

In [18]:
importances_df

,feature,feature_importance
0,Rate,0.534919
1,Slope,0.000069
2,HV,0.000060
3,Area,0.000049
4,CV,0.000016
5,SAR,-0.000033
6,Elevation,-0.000186
7,LC,-0.000489


In [ ]:
imp_features_df_all = pd.DataFrame(columns=["feature", "feature_importance", "site_test"])

for site_number in [1, 2, 3, 5,6,7,8,9,11,16,19,20,21,22,25,27,28,29,33,34,35,37]: # [1, 2, 3, 5,6,7,8,9,11,16,19,20,21,22,25,27,28,29,33,34,35,37]
    print("site: ", site_number)
    features, variable_to_predict = prediction.split_dataset_into_features_and_variable_to_predict(input_data_cleaned)
    features_train, features_test, variable_train, variable_test = prediction.get_train_and_test_data(features, variable_to_predict, site_number)
    prediction_model = train_random_forest_model(features_train, variable_train)
    variable_train_pred, variable_test_pred = predict_with_trained_model(prediction_model, features_train, features_test)
    subCatchment_numbers = prediction.get_subcatchment_numbers_for_a_site(site_number, variable_to_predict)
    liste_variable_test_HError = prediction.get_list_variable_test_Hind_Values(variable_test)
    liste_variable_test_pred_HError = variable_test_pred
    mse_test, r2_test = prediction.get_standard_quality_metrics(subCatchment_numbers, liste_variable_test_HError, liste_variable_test_pred_HError)
    importances_df = drop_col_feat_imp(prediction_model, features_train, variable_train.values.ravel(), random_state = 1)
    importances_df["site_test"] = site_number
    imp_features_df_all = pd.concat([imp_features_df_all, importances_df])
imp_features_df_all

In [20]:
imp_features_df_all.to_csv("output/" + "Feature_Importance_RF_SitesTests_clean.csv")